In [25]:
import warnings
warnings.filterwarnings('ignore')

In [28]:
import time
from datetime import datetime

from fastai.vision.all import *
import pyautogui
import pygetwindow as gw
import keyboard
import numpy as np
import cv2

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [76]:
# need to declare globally cuz to override label_func
path = 'test_dataset'
fnames = get_image_files(path)
def label_func(x): return x.parent.name

def load_model(path):    
    learn = load_learner(os.path.abspath(path))
    return learn

In [77]:
def launch_chrome_dino():
    try:
        options = Options()
        options.add_argument("--window-size=800,800")
        driver = webdriver.Chrome(executable_path='./chromedriver', options=options)
        driver.get('chrome://dino')
    except Exception as e:
#         print(e)
        pass
    finally:
        print('Launching Dino.')
        
    screen_size = (pyautogui.size().width, pyautogui.size().height)
    dino_window = gw.getWindowsWithTitle('chrome://dino/ - Google Chrome')[0]
    
    window_size = 800
    dino_window.moveTo(screen_size[0]-window_size, 0)
    dino_window.resizeTo(window_size, window_size)
    
    return driver, dino_window

In [82]:
def screenshot_frame():
    # screenshot
    frame = pyautogui.screenshot(region=(screen_size[0]-window_size+7, 0, window_size-14, window_size-14))

    # change to grayscale
    frame = cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2GRAY)
    
    # resize to compatible with resnet model
    frame = cv2.resize(frame, (224,224))
    
    # convert to black n white to count white n black pixel
    # for checking black or white background
    thresh = 107
    sample_frame = cv2.threshold(frame, thresh, 255, cv2.THRESH_BINARY)[1]
    pixel_count = np.unique(sample_frame, return_counts=True)[1]
    
    if pixel_count[1] > pixel_count[0]:
        thresh = 180
        # actually change to black n white
        frame = cv2.threshold(frame, thresh, 255, cv2.THRESH_BINARY)[1]
        frame = np.invert(frame)
        # remove top and bottom ui (change to plain)
        frame[:50,:] = 1
        frame[-50:,:] = 1
    else:
        thresh = 110
        frame = cv2.threshold(frame, thresh, 255, cv2.THRESH_BINARY)[1]
        frame[:50,:] = 0
        frame[-50:,:] = 0
    
    return frame

In [83]:
learn = load_model('models/v2.pkl')

In [84]:
frame = cv2.imread('dataset/up/04-14-2022-02-00-50-789821.png');
learn.predict(frame)[0]

'up'

In [90]:
dino_window = launch_chrome_dino()

#start the game
pyautogui.press('up')

start = time.time()
run = True

while run:
    frame = screenshot_frame()
    hotkeys = learn.predict(frame)[0]
    
    if hotkeys == 'up':
        pyautogui.press('up')
        print('Jump')
    elif hotkeys == 'down':
        pyautogui.press('down')
    
    end = time.time()
    time_elapsed = end - start
    if time_elapsed > 30:
        run = False

Launching Dino.


Jump


Jump
